In [1]:
import os
from pprint import pprint
from importlib import import_module

from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import joinedload, selectinload, subqueryload, Load, load_only
from sqlalchemy.orm.exc import NoResultFound
from sqlalchemy.dialects import postgresql

from dataservice.extensions import db
from dataservice import create_app
from dataservice.api.investigator.models import Investigator
from dataservice.api.study.models import Study
from dataservice.api.participant.models import Participant
from dataservice.api.family_relationship.models import FamilyRelationship
from dataservice.api.diagnosis.models import Diagnosis
from dataservice.api.outcome.models import Outcome
from dataservice.api.phenotype.models import Phenotype
from dataservice.api.sample.models import Sample
from dataservice.api.aliquot.models import Aliquot
from dataservice.api.sequencing_experiment.models import SequencingExperiment
from dataservice.api.genomic_file.models import GenomicFile
from dataservice.api.workflow.models import Workflow, WorkflowGenomicFile
from dataservice.api.study_file.models import StudyFile

from dataservice.util.data_import.utils import to_camel_case
from dataservice.util.data_import.etl.defaults import DEFAULT_ENTITY_TYPES

study_id = 'SD_PTCT829R'

class BaseLoader(object):

    def __init__(self, config_name=None):
        if not config_name:
            config_name = os.environ.get('FLASK_CONFIG', 'default')
        self.setup(config_name)
        self.entity_id_map = {}

    def setup(self, config_name):
        """
        Creates tables in database
        """
        self.app = create_app(config_name)
        self.app.config['SQLALCHEMY_ECHO'] = True
        self.app_context = self.app.app_context()
        self.app_context.push()
        db.create_all()
        self.import_models()

    def teardown(self):
        """
        Clean up
        """
        db.session.remove()
        self.app_context.pop()

    def drop_all(self, study_external_id):
        """
        Delete all data related to a study
        """
        from dataservice.api.study.models import Study
        from dataservice.api.investigator.models import Investigator

        try:
            study = Study.query.filter_by(external_id=study_external_id).one()
        except NoResultFound:
            print("Study {} not found. Aborting drop all for this dataset"
                  .format(study_external_id))
        else:
            # Save investigator id
            investigator_id = study.investigator_id

            # Delete study
            db.session.delete(study)

            # Delete investigator
            if investigator_id:
                investigator = Investigator.query.get(investigator_id)
                db.session.delete(investigator)

            db.session.commit()

    def import_models(self, skip_entities=[]):
        """
        Load all entities into db
        """
        # For each entity type
        for entity_type in DEFAULT_ENTITY_TYPES:
            # Skip some entities
            if entity_type in skip_entities:
                continue
            # Dynamically import entity model class
            model_name = to_camel_case(entity_type)
            model_module_path = 'dataservice.api.{}.models'.format(
                entity_type)
            models_module = import_module(model_module_path)
            model = getattr(models_module, model_name)

In [2]:
loader = BaseLoader()

2018-04-10 12:33:50,758 INFO sqlalchemy.engine.base.Engine select version()
2018-04-10 12:33:50,761 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 12:33:50,766 INFO sqlalchemy.engine.base.Engine select current_schema()
2018-04-10 12:33:50,767 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 12:33:50,772 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-10 12:33:50,773 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 12:33:50,775 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-10 12:33:50,777 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 12:33:50,780 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2018-04-10 12:33:50,781 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 12:33:50,784 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [ ]:
# Genomic files for study
q = (Participant.query.filter_by(study_id=study_id).options(
              selectinload(Participant.samples)
              .load_only('kf_id')
              .selectinload(Sample.aliquots)
              .load_only('kf_id')
              .selectinload(Aliquot.sequencing_experiments)
              .load_only('kf_id')
              .selectinload(SequencingExperiment.genomic_files)))
print(q.statement.compile(dialect=postgresql.dialect()))

In [ ]:
pts = q.all()

In [ ]:
for pt in pts:
    for sample in pt.samples:
        for aliquot in sample.aliquots:
            for seqexp in aliquot.sequencing_experiments:
                for gf in seqexp.genomic_files:
                        gf
#                     print(gf.file_name)

In [ ]:
q = (GenomicFile.query.options(
     joinedload(GenomicFile.sequencing_experiment).load_only("kf_id")
     .joinedload(SequencingExperiment.aliquot).load_only("kf_id")
     .joinedload(Aliquot.sample).load_only("kf_id"))
     .join(Sample.participant).options(Load(Participant).load_only("kf_id", "study_id"))
     .filter(Participant.study_id==study_id))
print(q.statement.compile(dialect=postgresql.dialect()))

In [ ]:
results = q.all()
for r in results:
    print(r.file_name)

In [ ]:
sql_q = \
"""
select genomic_file.uuid AS genomic_file_uuid, genomic_file.created_at AS genomic_file_created_at, genomic_file.modified_at AS genomic_file_modified_at, genomic_file.file_name AS genomic_file_file_name, genomic_file.data_type AS genomic_file_data_type, genomic_file.file_format AS genomic_file_file_format, genomic_file.file_size AS genomic_file_file_size, genomic_file.file_url AS genomic_file_file_url, genomic_file.md5sum AS genomic_file_md5sum, genomic_file.controlled_access AS genomic_file_controlled_access, genomic_file.sequencing_experiment_id AS genomic_file_sequencing_experiment_id, genomic_file.kf_id AS genomic_file_kf_id  from genomic_file
inner join sequencing_experiment on genomic_file.sequencing_experiment_id=sequencing_experiment.kf_id
inner join aliquot on sequencing_experiment.aliquot_id=aliquot.kf_id
inner join sample on aliquot.sample_id=sample.kf_id
inner join participant on sample.participant_id=participant.kf_id
where participant.study_id={};
""".format(study_id)
results = db.session.query(GenomicFile).from_statement(sql_q)
for r in results:
    print(r.file_name)

In [ ]:
db.session.rollback()

In [28]:
q = (Diagnosis.query.options(joinedload(Diagnosis.participant, innerjoin=True).load_only('kf_id'))
# .options(Load(Participant).load_only('kf_id', 'study_id'))
.filter(Participant.study_id == study_id))
print(q.statement.compile(dialect=postgresql.dialect()))

SELECT diagnosis.uuid, diagnosis.created_at, diagnosis.modified_at, diagnosis.external_id, diagnosis.diagnosis, diagnosis.diagnosis_category, diagnosis.tumor_location, diagnosis.age_at_event_days, diagnosis.participant_id, diagnosis.kf_id, participant_1.kf_id 
FROM participant, diagnosis JOIN participant AS participant_1 ON participant_1.kf_id = diagnosis.participant_id 
WHERE participant.study_id = %(study_id_1)s


/Users/singhn4/backup-kf-api-dataservice/venv/lib/python3.6/site-packages/sqlalchemy/sql/base.py:528: SAWarning: Column 'kf_id' on table <sqlalchemy.sql.selectable.Select at 0x111d9c390; Select object> being replaced by Column('kf_id', KfId(length=11), table=<Select object>, primary_key=True, nullable=False), which has the same key.  Consider use_labels for select() statements.
  (key, getattr(existing, 'table', None), value))


In [26]:
q = (Diagnosis.query
     .join(Participant.diagnoses)
     .options(Load(Participant).load_only('kf_id', 'study_id'))
     .filter(Participant.study_id == study_id))
print(q.statement.compile(dialect=postgresql.dialect()))

SELECT diagnosis.uuid, diagnosis.created_at, diagnosis.modified_at, diagnosis.external_id, diagnosis.diagnosis, diagnosis.diagnosis_category, diagnosis.tumor_location, diagnosis.age_at_event_days, diagnosis.participant_id, diagnosis.kf_id 
FROM participant JOIN diagnosis ON participant.kf_id = diagnosis.participant_id 
WHERE participant.study_id = %(study_id_1)s


In [24]:
# q = (Diagnosis.query
#      .options(joinedload(Participant.diagnoses).load_only('kf_id')))
#      .options(Load(Participant).load_only('kf_id', 'study_id'))
#      .filter(Participant.study_id == study_id))
# print(q.statement.compile(dialect=postgresql.dialect()))

### Wrong way to load children through joins

In [6]:
q = (GenomicFile.query.options(
     joinedload(GenomicFile.sequencing_experiment).load_only("kf_id")
     .joinedload(SequencingExperiment.aliquot).load_only("kf_id")
     .joinedload(Aliquot.sample).load_only("kf_id"))
     .join(Sample.participant).options(Load(Participant).load_only("kf_id", "study_id"))
     .filter(Participant.study_id==study_id))
print(q.statement.compile(dialect=postgresql.dialect()))

SELECT genomic_file.uuid, genomic_file.created_at, genomic_file.modified_at, genomic_file.file_name, genomic_file.data_type, genomic_file.file_format, genomic_file.file_size, genomic_file.file_url, genomic_file.md5sum, genomic_file.controlled_access, genomic_file.sequencing_experiment_id, genomic_file.kf_id, sequencing_experiment_1.kf_id, aliquot_1.kf_id, sample_1.kf_id 
FROM sample JOIN participant ON participant.kf_id = sample.participant_id, genomic_file LEFT OUTER JOIN sequencing_experiment AS sequencing_experiment_1 ON sequencing_experiment_1.kf_id = genomic_file.sequencing_experiment_id LEFT OUTER JOIN aliquot AS aliquot_1 ON aliquot_1.kf_id = sequencing_experiment_1.aliquot_id LEFT OUTER JOIN sample AS sample_1 ON sample_1.kf_id = aliquot_1.sample_id 
WHERE participant.study_id = %(study_id_1)s


/Users/singhn4/backup-kf-api-dataservice/venv/lib/python3.6/site-packages/sqlalchemy/sql/base.py:528: SAWarning: Column 'kf_id' on table <sqlalchemy.sql.selectable.Select at 0x1118fe630; Select object> being replaced by Column('kf_id', KfId(length=11), table=<Select object>, primary_key=True, nullable=False), which has the same key.  Consider use_labels for select() statements.
  (key, getattr(existing, 'table', None), value))


### Correct way to load children through joins

In [15]:
q = (GenomicFile.query
     .join(SequencingExperiment.genomic_files)
     .join(Aliquot.sequencing_experiments)
     .join(Sample.aliquots)
     .join(Participant.samples)
     .filter(Participant.study_id==study_id)
    )
print(q.statement.compile(dialect=postgresql.dialect()))

SELECT genomic_file.uuid, genomic_file.created_at, genomic_file.modified_at, genomic_file.file_name, genomic_file.data_type, genomic_file.file_format, genomic_file.file_size, genomic_file.file_url, genomic_file.md5sum, genomic_file.controlled_access, genomic_file.sequencing_experiment_id, genomic_file.kf_id 
FROM sequencing_experiment JOIN genomic_file ON sequencing_experiment.kf_id = genomic_file.sequencing_experiment_id, aliquot JOIN sequencing_experiment ON aliquot.kf_id = sequencing_experiment.aliquot_id, sample JOIN aliquot ON sample.kf_id = aliquot.sample_id, participant JOIN sample ON participant.kf_id = sample.participant_id 
WHERE participant.study_id = %(study_id_1)s


In [17]:
# Participants
q = (Participant.query
                .options(joinedload(Participant.diagnoses)
                        .load_only('kf_id'))
                .options(joinedload(Participant.samples)
                        .load_only('kf_id'))
                .options(joinedload(Participant.phenotypes)
                        .load_only('kf_id'))
                .options(joinedload(Participant.outcomes)
                        .load_only('kf_id')))
print(q.statement.compile(dialect=postgresql.dialect()))

SELECT participant.uuid, participant.created_at, participant.modified_at, participant.external_id, participant.family_id, participant.is_proband, participant.consent_type, participant.race, participant.ethnicity, participant.gender, participant.study_id, participant.kf_id, diagnosis_1.kf_id, sample_1.kf_id, outcome_1.kf_id, phenotype_1.kf_id 
FROM participant LEFT OUTER JOIN diagnosis AS diagnosis_1 ON participant.kf_id = diagnosis_1.participant_id LEFT OUTER JOIN sample AS sample_1 ON participant.kf_id = sample_1.participant_id LEFT OUTER JOIN outcome AS outcome_1 ON participant.kf_id = outcome_1.participant_id LEFT OUTER JOIN phenotype AS phenotype_1 ON participant.kf_id = phenotype_1.participant_id


/Users/singhn4/backup-kf-api-dataservice/venv/lib/python3.6/site-packages/sqlalchemy/sql/base.py:528: SAWarning: Column 'kf_id' on table <sqlalchemy.sql.selectable.Select at 0x111ddb4a8; Select object> being replaced by Column('kf_id', KfId(length=11), table=<Select object>, primary_key=True, nullable=False), which has the same key.  Consider use_labels for select() statements.
  (key, getattr(existing, 'table', None), value))
